In [13]:
from clean2 import *

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [16]:
df = data_wrangling()
tmp = df.groupby(
        ["C/A", "UNIT", "SCP", "STATION", "DATE", "AMPM", "DAY_NAME"], as_index=False
    ).ENTRIES.max()

##
t = df.groupby(
    ["C/A", "UNIT", "SCP", "STATION", "DATE", "AMPM", "DAY_NAME"], as_index=False
).EXITS.max()
tmp["EXITS"] = t["EXITS"]

# create prev_date and prev_entries cols by shifting these columns forward one day
# if shifting date and entries, don't group by date
tmp[["PREV_DATE", "PREV_ENTRIES", "PREV_EXITS"]] = tmp.groupby(
    ["C/A", "UNIT", "SCP", "STATION", "AMPM"]
)[["DATE", "ENTRIES", "EXITS"]].apply(lambda grp: grp.shift(1))

# Drop the rows for the earliest date in the df, which are now NaNs for prev_date and prev_entries cols
tmp.dropna(subset=["PREV_DATE"], axis=0, inplace=True)

tmp = get_tmp_entries(df)

In [4]:
df.sample(10, random_state=10)

,DATE_TIME,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,EXITS,AMPM,DAY_NAME
178257,2016-09-08 12:00:00,R527,R122,00-03-03,90 ST-ELMHURST,7,IRT,09/08/2016,12:00:00,1234777,400728,PM,Thursday
77707,2016-09-08 13:00:00,N310,R140,01-06-01,QUEENS PLAZA,EMR,IND,09/08/2016,13:00:00,1128183,1135648,PM,Thursday
36439,2016-09-14 13:00:00,J007,R377,00-00-02,FLUSHING AV,JM,BMT,09/14/2016,13:00:00,2817479,504020,PM,Wednesday
21260,2016-09-04 12:00:00,C012,R258,01-03-01,4AV-9 ST,DFGMNR,BMT,09/04/2016,12:00:00,1622601,533083,PM,Sunday
141285,2016-09-04 06:00:00,R204A,R043,03-06-01,WALL ST,45,IRT,09/04/2016,06:00:00,1333815,2971917,AM,Sunday
178758,2016-09-08 00:00:00,R528,R097,00-05-01,JUNCTION BLVD,7,IRT,09/08/2016,00:00:00,3414515,3813419,AM,Thursday
69677,2016-09-11 04:22:00,N203,R195,00-03-00,161/YANKEE STAD,BD4,IND,09/11/2016,04:22:00,5301848,2094275,AM,Sunday
144507,2016-09-11 17:00:00,R227A,R131,01-06-00,23 ST,6,IRT,09/11/2016,17:00:00,2919630,2769335,PM,Sunday
176134,2016-09-06 16:00:00,R520,R223,01-05-00,46 ST BLISS ST,7,IRT,09/06/2016,16:00:00,1734,0,PM,Tuesday
78516,2016-08-30 21:00:00,N316A,R267,01-03-01,46 ST,MR,IND,08/30/2016,21:00:00,1923047,6149191,PM,Tuesday


In [17]:
tmp

,C/A,UNIT,SCP,STATION,DATE,AMPM,DAY_NAME,ENTRIES,EXITS,PREV_DATE,PREV_ENTRIES,PREV_EXITS,TMP_ENTRIES,TMP_EXITS
2,A002,R051,02-00-00,59 ST,08/28/2016,AM,Sunday,5800295,1966317,08/27/2016,5799492.0,1966079.0,803.0,238.0
3,A002,R051,02-00-00,59 ST,08/28/2016,PM,Sunday,5800798,1966512,08/27/2016,5800121.0,1966271.0,677.0,241.0
4,A002,R051,02-00-00,59 ST,08/29/2016,AM,Monday,5800996,1966657,08/28/2016,5800295.0,1966317.0,701.0,340.0
5,A002,R051,02-00-00,59 ST,08/29/2016,PM,Monday,5802336,1967033,08/28/2016,5800798.0,1966512.0,1538.0,521.0
6,A002,R051,02-00-00,59 ST,08/30/2016,AM,Tuesday,5802575,1967164,08/29/2016,5800996.0,1966657.0,1579.0,507.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191888,TRAM2,R469,00-05-01,RIT-ROOSEVELT,09/14/2016,PM,Wednesday,5554,271,09/13/2016,5554.0,271.0,0.0,0.0
191889,TRAM2,R469,00-05-01,RIT-ROOSEVELT,09/15/2016,AM,Thursday,5554,271,09/14/2016,5554.0,271.0,0.0,0.0
191890,TRAM2,R469,00-05-01,RIT-ROOSEVELT,09/15/2016,PM,Thursday,5554,271,09/14/2016,5554.0,271.0,0.0,0.0
191891,TRAM2,R469,00-05-01,RIT-ROOSEVELT,09/16/2016,AM,Friday,5554,271,09/15/2016,5554.0,271.0,0.0,0.0


In [23]:
tmp['traffic'] = tmp.TMP_ENTRIES + tmp.TMP_EXITS
AMPM = ['AM', 'PM']

# TOP TEN AM
mask = tmp.AMPM == 'AM'
tmp[mask].groupby('STATION').agg('mean').sort_values(['traffic']).head(10)

,ENTRIES,EXITS,PREV_ENTRIES,PREV_EXITS,TMP_ENTRIES,TMP_EXITS,traffic
STATION,,,,,,,
SARATOGA AV,5.055697e+06,3.734688e+06,5.055697e+06,3.734686e+06,0.250000,2.020000,2.270000
PENNSYLVANIA AV,3.722705e+06,6.499313e+05,3.901245e+06,6.755982e+05,14.642857,7.400000,22.042857
ORCHARD BEACH,9.424614e+04,5.376222e+03,9.422459e+04,5.374822e+03,21.555556,1.400000,22.955556
TOMPKINSVILLE,1.291279e+06,3.543617e+05,1.291182e+06,3.543617e+05,96.442857,0.000000,96.442857
BROAD CHANNEL,6.076940e+05,2.082776e+05,6.076010e+05,2.082480e+05,93.033333,29.566667,122.600000
BEACH 105 ST,2.372695e+07,3.322571e+05,2.372688e+07,3.321856e+05,71.220000,71.520000,142.740000
BEACH 44 ST,5.374742e+05,6.913872e+05,5.373444e+05,6.912432e+05,129.802083,144.093750,273.895833
BEACH 98 ST,8.977479e+05,9.745233e+05,8.975987e+05,9.743920e+05,149.260000,131.250000,280.510000
E 143/ST MARY'S,8.000172e+05,8.927925e+05,7.998783e+05,8.926396e+05,138.883333,152.916667,291.800000


In [24]:
# TOP TEN AM
mask = tmp.AMPM == 'AM'
tmp[mask].groupby('STATION').agg('mean').sort_values(['traffic']).head(10)

,ENTRIES,EXITS,PREV_ENTRIES,PREV_EXITS,TMP_ENTRIES,TMP_EXITS,traffic
STATION,,,,,,,
SARATOGA AV,5.055697e+06,3.734688e+06,5.055697e+06,3.734686e+06,0.250000,2.020000,2.270000
PENNSYLVANIA AV,3.722705e+06,6.499313e+05,3.901245e+06,6.755982e+05,14.642857,7.400000,22.042857
ORCHARD BEACH,9.424614e+04,5.376222e+03,9.422459e+04,5.374822e+03,21.555556,1.400000,22.955556
TOMPKINSVILLE,1.291279e+06,3.543617e+05,1.291182e+06,3.543617e+05,96.442857,0.000000,96.442857
BROAD CHANNEL,6.076940e+05,2.082776e+05,6.076010e+05,2.082480e+05,93.033333,29.566667,122.600000
BEACH 105 ST,2.372695e+07,3.322571e+05,2.372688e+07,3.321856e+05,71.220000,71.520000,142.740000
BEACH 44 ST,5.374742e+05,6.913872e+05,5.373444e+05,6.912432e+05,129.802083,144.093750,273.895833
BEACH 98 ST,8.977479e+05,9.745233e+05,8.975987e+05,9.743920e+05,149.260000,131.250000,280.510000
E 143/ST MARY'S,8.000172e+05,8.927925e+05,7.998783e+05,8.926396e+05,138.883333,152.916667,291.800000


In [25]:
# TOP TEN PM
mask = tmp.AMPM == 'PM'
tmp[mask].groupby('STATION').agg('mean').sort_values(['traffic']).head(10)

,ENTRIES,EXITS,PREV_ENTRIES,PREV_EXITS,TMP_ENTRIES,TMP_EXITS,traffic
STATION,,,,,,,
SARATOGA AV,5.055697e+06,3.734690e+06,5.055697e+06,3.734688e+06,0.250000,2.230000,2.480000
ORCHARD BEACH,9.425747e+04,5.377067e+03,9.424613e+04,5.376222e+03,11.333333,0.844444,12.177778
PENNSYLVANIA AV,3.569731e+06,6.232343e+05,3.740935e+06,6.478462e+05,14.041096,7.342466,21.383562
TOMPKINSVILLE,1.291340e+06,3.543617e+05,1.291242e+06,3.543617e+05,98.071429,0.000000,98.071429
BROAD CHANNEL,6.077318e+05,2.082995e+05,6.076385e+05,2.082700e+05,93.283333,29.500000,122.783333
BEACH 105 ST,2.372699e+07,3.323094e+05,2.372693e+07,3.322421e+05,65.560000,67.290000,132.850000
BEACH 98 ST,8.978409e+05,9.746149e+05,8.977040e+05,9.744936e+05,136.890000,121.320000,258.210000
BEACH 44 ST,5.375518e+05,6.914724e+05,5.374207e+05,6.913267e+05,131.052083,145.729167,276.781250
E 143/ST MARY'S,8.001181e+05,8.928787e+05,7.999755e+05,8.927237e+05,142.658333,155.000000,297.658333
